In [1]:
import numpy as np
import functools
import h5py
from keras.models import load_model
import cv2
import torch
from PIL import Image

predictions = []
def open_file(file):
   with h5py.File(file, 'r') as hf:
        #get the data
        a_group_key = list(hf.keys())[0]
        test = list(hf[a_group_key])



        # transform to appropriate numpy array
        test = test[0:]
        test = np.stack(test, axis=0)
        return test

TIMESTEPS_X = 13
X_Seq = TIMESTEPS_X * 3

def data_preprocessing(data, COVID):

   #combine dimensions of 288 and 8 as in the CNN paper
   def combine_dims(a, i=0, n=1):
     """
     Combines dimensions of numpy array `a`,
     starting at index `i`,
     and combining `n` dimensions
     """
     s = list(a.shape)
     combined = functools.reduce(lambda x,y: x*y, s[i:i+n+1])
     return np.reshape(a, s[:i] + [combined] + s[i+n+1:])
    
    
    
   print(data.shape)
   data = data.swapaxes(0, 4)
   print(data.shape)
   data = combine_dims(data)

   print(data.shape)
   #divide the data by 255
   data = np.array(data)
   data = data / 255.
   data = data.swapaxes(0, 3)
   print(data.shape)
   
   return data

In [50]:
PATH = "data/raw/"
CITIES = ["BERLIN","CHICAGO","ISTANBUL","MELBOURNE"]
#model = load_model('BERLIN.h5', compile=False)
city='MELBOURNE'
#for city in CITIES:
data = open_file('data/raw/{city}/{city}_test_temporal.h5'.format(city=city))
part_1 = data[0:25]
part_2 = data[25:50]
part_3 = data[50:75]
part_4 = data[75:100]
hf = h5py.File(city+'_1.h5', 'w')
hf.create_dataset('dataset_1', data=part_1)
hf = h5py.File(city+'_2.h5', 'w')
hf.create_dataset('dataset_1', data=part_2)
hf = h5py.File(city+'_3.h5', 'w')
hf.create_dataset('dataset_1', data=part_3)
hf = h5py.File(city+'_4.h5', 'w')
hf.create_dataset('dataset_1', data=part_4)
hf.close()

In [6]:
from keras.utils.vis_utils import plot_model
PATH = "data/raw/"
CITIES = ["BERLIN","CHICAGO","ISTANBUL","MELBOURNE"]
model = load_model('MELBOURNE.h5', compile=False)

In [5]:
!pip install graphviz

You should consider upgrading via the 'c:\users\oli\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [7]:
plot_model(model, to_file="model.png", show_shapes=True, show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [52]:
#5,10,15,30,45 and 60
predictions=[]
city='MELBOURNE'
#for city in CITIES:
for i in range(1,5):
    data = open_file('{city}_{i}.h5'.format(city=city, i=i))
    x = data_preprocessing(data, True)
    print(x.shape)
    for hour in x:
        hour_prediction = []
        prediction = model.predict(np.expand_dims(hour, axis = 0))
        prediction = prediction[:, 1:, 6:442, :]

        pred_5_10_15 = prediction[0, :, :, :24]
        pred_30 = prediction[0, :, :, 40:]

        new_data = hour[:, :, :48]
        new_data = np.append(new_data, prediction[0], axis = -1)

        prediction = model.predict(np.expand_dims(new_data, axis = 0))
        prediction = prediction[:, 1:, 6:442, :]

        pred_45 = prediction[0, :, :, 16:24]
        pred_60 = prediction[0, :, :, 40:]

        hour_prediction = np.append(pred_5_10_15, pred_30, axis = -1)
        hour_prediction = np.append(hour_prediction, pred_45, axis = -1)
        hour_prediction = np.append(hour_prediction, pred_60, axis = -1)
        hour_prediction = hour_prediction *255.
        predictions.append(hour_prediction)
    print(len(predictions))

(25, 12, 495, 436, 8)
(8, 12, 495, 436, 25)
(96, 495, 436, 25)
(25, 495, 436, 96)
(25, 495, 436, 96)
25
(25, 12, 495, 436, 8)
(8, 12, 495, 436, 25)
(96, 495, 436, 25)
(25, 495, 436, 96)
(25, 495, 436, 96)
50
(25, 12, 495, 436, 8)
(8, 12, 495, 436, 25)
(96, 495, 436, 25)
(25, 495, 436, 96)
(25, 495, 436, 96)
75
(25, 12, 495, 436, 8)
(8, 12, 495, 436, 25)
(96, 495, 436, 25)
(25, 495, 436, 96)
(25, 495, 436, 96)
100


In [53]:
predictions = np.array(predictions)
predictions = np.moveaxis(predictions, -1, 1)
predictions.shape

(100, 48, 495, 436)

In [54]:
"""
`(k, 12 * 8, 495, 436) -> (k, 12, 495, 436, 8)`
"""
# `(12, 495, 436, 8) -> (1, 12, 495, 436, 8)`
data = torch.unsqueeze(torch.from_numpy(predictions), 0)
print(data.shape)
num_time_steps = int(predictions.shape[1] / 8)
# (k, 12 * 8, 495, 436) -> (k, 12, 8, 495, 436)
data = torch.reshape(data, (100, 6, 8, 495, 436))

# (k, 12, 8, 495, 436) -> (k, 12, 495, 436, 8)
data = torch.moveaxis(data, 2, 4)

# `(1, 12, 495, 436, 8) -> (12, 495, 436, 8)`
data = torch.squeeze(data, 0)
print(data.shape)

torch.Size([1, 100, 48, 495, 436])
torch.Size([100, 6, 495, 436, 8])


In [55]:
hf = h5py.File('MELBOURNE_test_temporal.h5', 'w')
hf.create_dataset('dataset_1', data=data, compression="gzip", compression_opts=6)
hf.close()

In [56]:
sa = open_file('BERLIN_test_temporal.h5')

In [57]:
sa.shape

(100, 6, 495, 436, 8)